In [1]:
import os
from langchain_openai import OpenAIEmbeddings
from pinecone import Pinecone
from tqdm.autonotebook import tqdm
from langchain_openai import ChatOpenAI
from langchain.vectorstores import Pinecone as LPPinecone
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain.chains.conversation.memory import ConversationBufferWindowMemory



C:\Users\Fiberta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = os.environ.get('PINECONE_API_KEY') or '3e81b5e5-edfe-49bf-a6ee-7ff0a9d47ca2'

# configure client
pc = Pinecone(api_key=api_key)

model_name = 'text-embedding-ada-002'
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=os.environ['OPENAI_API_KEY']
)

index_name = 'pilot'
index = pc.Index(index_name)

text_field = "text"  # the metadata field that contains our text
vectorstore = LPPinecone(
    index, embed.embed_query, text_field
)

chat = ChatOpenAI(
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    model_name='gpt-3.5-turbo',
    temperature=0.0
)

# conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)
qa_with_sources = RetrievalQAWithSourcesChain.from_chain_type(
    llm=chat,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)




C:\Users\Fiberta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.vectorstores.pinecone.Pinecone` was deprecated in langchain-community 0.0.18 and will be removed in 0.2.0. An updated version of the class exists in the langchain-pinecone package and should be used instead. To use it run `pip install -U langchain-pinecone` and import as `from langchain_pinecone import Pinecone`.
  warn_deprecated(
C:\Users\Fiberta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_community\vectorstores\pinecone.py:68: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [3]:
from langchain.agents import Tool

tools = [
    Tool(
        name='Knowledge Base',
        func=qa_with_sources,
        description=(
            'use this tool when answering general knowledge queries to get '
            'more information about the topic'
        )
    )
]

In [4]:
from langchain.agents import initialize_agent

agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=chat,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory
)

C:\Users\Fiberta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [5]:
query = "Who is bill gates?"
agent(query)

C:\Users\Fiberta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
```json
{
    "action": "Knowledge Base",
    "action_input": "Bill Gates"
}
```
Observation: {'question': 'Bill Gates', 'answer': "I don't know.\n", 'sources': ''}
Thought:```json
{
    "action": "Final Answer",
    "action_input": "Bill Gates is a co-founder of Microsoft and a prominent figure in the technology industry."
}
```

> Finished chain.


{'input': 'Who is bill gates?',
 'chat_history': [],
 'output': 'Bill Gates is a co-founder of Microsoft and a prominent figure in the technology industry.'}

In [6]:
from langchain.callbacks import get_openai_callback

def count_tokens(agent, query):
    with get_openai_callback() as cb:
        result = agent(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

In [7]:

result = count_tokens(
    agent,
    "What's the result of an investment of $10,000 growing at 8% annually for 5 years with compound interest?"
)



> Entering new AgentExecutor chain...
```json
{
    "action": "Knowledge Base",
    "action_input": "Calculate the compound interest on an investment of $10,000 growing at 8% annually for 5 years"
}
```
Observation: {'question': 'Calculate the compound interest on an investment of $10,000 growing at 8% annually for 5 years', 'answer': "I don't know the answer.\n", 'sources': ''}
Thought:```json
{
    "action": "Final Answer",
    "action_input": "The tool was unable to provide the answer to the compound interest calculation on an investment of $10,000 growing at 8% annually for 5 years."
}
```

> Finished chain.
Spent a total of 4027 tokens


# New test example


In [8]:
# from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


In [9]:

# MEMORY_KEY = "chat_history"
# prompt = ChatPromptTemplate.from_messages(
#     [
#         (
#             "system",
#             "You are very powerful assistant, but bad at calculating lengths of words.",
#         ),
#         MessagesPlaceholder(variable_name=MEMORY_KEY),
#         ("user", "{input}"),
#         MessagesPlaceholder(variable_name="agent_scratchpad"),
#     ]
# )

In [10]:
# from langchain_core.messages import AIMessage, HumanMessage

# chat_history = []

In [11]:
# from langchain.agents import Tool

# tools = [
#     Tool(
#         name='Knowledge Base',
#         func=qa_with_sources,
#         description=(
#             'use this tool when answering general knowledge queries to get '
#             'more information about the topic'
#         )
#     )
# ]

In [12]:
# from langchain.agents.format_scratchpad.openai_tools import (
#     format_to_openai_tool_messages)
# from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
# from langchain.agents import AgentExecutor


# agent = (
#     {
#         "input": lambda x: x["input"],
#         "agent_scratchpad": lambda x: format_to_openai_tool_messages(
#             x["intermediate_steps"]
#         ),
#         "chat_history": lambda x: x["chat_history"],
#     }
#     | prompt
#     | tools
#     | OpenAIToolsAgentOutputParser()
# )
# agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)